In [1]:
import sys

sys.path.append('../src/d00_utils/')
from db_utils import dbReadWriteClean, dbReadWriteViews

In [2]:
io_clean = dbReadWriteClean()
df_dcm = io_clean.get_table('meta_lite')
df_dcm.head()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,dirname,filename,tag1,tag2,value,tags
0,72327,a_E45PRVD1,28,0010,34,"(0028,0010)"
1,72327,a_E45PRVD1,28,0011,36,"(0028,0011)"
2,72327,a_E45PRVD3,2,0010,1.2.840.10008.1.2.4.50,"(0002,0010)"
3,72327,a_E45PRVD3,8,0016,1.2.840.10008.5.1.4.1.1.6.1,"(0008,0016)"
4,72327,a_E45PRVD3,8,002a,20151106113728.000,"(0008,002a)"


In [3]:
print(df_dcm.shape)

(12493449, 6)


In [4]:
io_views = dbReadWriteViews()
df_inst_all = io_views.get_table('instances_w_labels')

# Dataframe edits to match with other table
df_inst = df_inst_all
df_inst.rename(columns={"instancefilename": "filename"}, inplace=True)
df_inst["filename"] = df_inst['filename'].str.rstrip()
df_inst["filename"] = df_inst['filename'].str.slice_replace(stop=0,repl='a_')
print(df_inst.shape)
df_inst.head()

(32554, 5)


,sopinstanceuid,filename,instanceidk,view,studyidk
0,1.2.840.113663.1500.1.365900332.3.3.20120703.9...,a_8Z0BWWL5,696108,plax,39531
1,1.2.840.113663.1500.1.365900332.3.4.20120703.9...,a_8Z0BWX0M,696109,plax,39531
2,1.2.840.113663.1500.1.365900332.3.7.20120703.9...,a_8Z0BWZGH,696112,a4c,39531
3,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,a_8Z0BXKCB,696140,plax,39533
4,1.2.840.113663.1500.1.365900332.3.14.20120703....,a_8Z0BXYUW,696157,plax,39533


In [5]:
# # Check to see how many instancefilenames match up
fn_list = df_inst['filename'].tolist()
# fn_list = [s.rstrip() for s in fn_list]
# fn_list_a = ['a_' + s for s in fn_list]

fn_list_dcm = df_dcm['filename'].tolist()
print(len(set(fn_list) & set(fn_list_dcm)))

29853


In [6]:
# Check to see how many studies line up; note - not unique!
dir_list = df_dcm['dirname'].tolist()
study_list = df_inst['studyidk'].tolist()
print(len(set(dir_list) & set(study_list)))

11721


In [7]:
merge_df = df_inst.merge(df_dcm, on ='filename')
merge_df.drop(columns=['sopinstanceuid', 'dirname', 'tags'], inplace=True)
print(merge_df.shape)
merge_df.head()

(472594, 7)


,filename,instanceidk,view,studyidk,tag1,tag2,value
0,a_8Z0BWWL5,696108,plax,39531,2,0010,1.2.840.10008.1.2.5
1,a_8Z0BWWL5,696108,plax,39531,8,0012,20120703
2,a_8Z0BWWL5,696108,plax,39531,8,0016,1.2.840.10008.5.1.4.1.1.6.1
3,a_8Z0BWWL5,696108,plax,39531,8,002a,20120703090118
4,a_8Z0BWWL5,696108,plax,39531,8,1090,iE33


In [13]:
# Get only tags that we care about
df = merge_df
df['tag1'] = df['tag1'].astype(str) # consistency with tag2

# Region Spatial Format: (0018,6012) - no values entered throughout db
#df_region_sfa = df.loc[(df['tag1'] == '18') & (df['tag2'] == '6012')] 
df_num_frames = df.loc[(df['tag1'] == '28') & (df['tag2'] == '0008')] 
df_ultra_color = df.loc[(df['tag1'] == '28') & (df['tag2'] == '0014')]

In [19]:
#print(df_region_sfa.shape)
frame_vals = df_num_frames['value'].tolist()
print('Minimum number of frames: ' + str(min(frame_vals)))
#df_num_frames['value'].value_counts()

import numpy as np
print('Any empties in df? No')
np.where(df_num_frames.applymap(lambda x: x==''))

Minimum number of frames: 10
Any empties in df? No


(array([], dtype=int64), array([], dtype=int64))

In [15]:
# Filter out tag values we are not interested in, e.g.

# Number of Frames (0028,0008) should be >1
## WAITING ON: new meta_lite from WT
# Before WT's new meta_lite, no instances with 1 frame
# run cell above to see if frame_vals < 1
#df_num_frames.dtypes
df_num_frames = df_num_frames[df_num_frames['value'] != '1']
df_num_frames.head()

,filename,instanceidk,view,studyidk,tag1,tag2,value
27,a_8Z0BWX0M,696109,plax,39531,28,0008,69
43,a_8Z0BWZGH,696112,a4c,39531,28,0008,23
58,a_8ZGVB44R,696422,plax,39552,28,0008,72
73,a_8ZGVB5TY,696426,a4c,39552,28,0008,76
88,a_8ZGVB7C1,696430,a2c,39552,28,0008,75


In [10]:
# Ultrasound Color (0028,0014) should be '00'
## WAITING ON: new meta_lite from WT
print(df_ultra_color['value'].value_counts()) # should be 00's, 01's
df_ultra_color = df_ultra_color[df_ultra_color['value'] == '00']

Series([], Name: value, dtype: int64)


In [11]:
df_tags = df_num_frames.append(df_ultra_color)
print(df_tags.shape)
df_tags.head()

(25656, 7)


,filename,instanceidk,view,studyidk,tag1,tag2,value
27,a_8Z0BWX0M,696109,plax,39531,28,0008,69
43,a_8Z0BWZGH,696112,a4c,39531,28,0008,23
58,a_8ZGVB44R,696422,plax,39552,28,0008,72
73,a_8ZGVB5TY,696426,a4c,39552,28,0008,76
88,a_8ZGVB7C1,696430,a2c,39552,28,0008,75


In [12]:
# Now we have df_tags, which is all instances filtered by tag values
# Use this to keep desired rows in original instances_w_labels table
good_instances = list(set(df_tags['instanceidk'].tolist()))
print('Number of good instances: ' + str(len(good_instances)))

df = df_inst_all
df = df[df['instanceidk'].isin(good_instances)]
print(df.shape)
#io_views.save_to_db(df, 'instances_w_labels')

# ^^^ Note this overwrites original instances_w_labels
# When done with nb, add this script at end of filter_views.py

Number of good instances: 25407
(25407, 5)
